<a href="https://colab.research.google.com/github/yorkjong/rs_rating/blob/main/notebooks/ibd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install and Setup

#### Install Packages

In [1]:
%pip install "git+https://github.com/yorkjong/rs_rating.git"
%pip install requests-cache

  Cloning https://github.com/yorkjong/rs_rating.git to /tmp/pip-req-build-r4oojf7l
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/rs_rating.git /tmp/pip-req-build-r4oojf7l
  Resolved https://github.com/yorkjong/rs_rating.git to commit c0e534ff58c12b1ed214f164a8b8983e2abc70de
  Preparing metadata (setup.py) ... done
  Created wheel for rs_rating: filename=rs_rating-1.0-py3-none-any.whl size=24578 sha256=04081f7aa44d6c61ed27d0423c2254d62957e38020e65525aab08ef6e4d73c5e
  Stored in directory: /tmp/pip-ephem-wheel-cache-iffp01_2/wheels/6c/e3/e2/d823f25048beb6ccc0ebc402d5210ce344ec7a829a17558710
Successfully built rs_rating
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.4 MB/s eta 0:00:00


#### Setup and Configuration

In [2]:
# @title Enable DataFrame Formatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [4]:
# @title Rank Function of IBD RS Rating

import os
from datetime import datetime

from rs_rating import ibd_rs
from rs_rating.stock_indices import get_tickers

def remove_failed_tickers(tickers):
    delisted = ['BRK.B', 'LEN.B', 'BF.B', 'UHAL.B', 'BF.A', 'CWEN.A', 'HEI.A']
    invalid = ['GEV', 'SOLV', 'VLTO', 'SW', 'ARM', 'CART', 'AS', 'BIRK', 'VSTS','LOAR', 'ALAB','GRAL', 'SEG']
    invalid += ['00945B.TW', '6928.TW', '6914.TW', '6771.TW', '00944.TW', '8162.TW', '1563.TW', '00946.TW', '00941.TW', '6423.TW', '00940.TW', '00939.TW', '4949.TW', '00943.TW', '8487.TW', '6794.TW', '6949.TW', '4771.TW']
    invalid += ['00936.TW', '6805.TW', '2254.TW', '6658.TW', '00935.TW', '6592B.TW', '6526.TW', '6906.TW', '4736.TW', '00636K.TW', '6968.TWO', '4442.TWO', '6534.TW', '6901.TW', '00934.TW', '00657K.TW', '6472.TW', '2258.TW', '6916.TW', '2762.TW', '6933.TW']
    invalid += ['02001R.TW', '020031.TW', '020039.TW', '020016.TW', '02001L.TW', '020019.TW', '020028.TW', '020020.TW', '02001S.TW', '020018.TW', '020038.TW', '020034.TW', '020011.TW', '020030.TW', '020012.TW', '020036.TW', '020029.TW', '020000.TW', '020015.TW', '020037.TW']
    invalid += ['6890.TW', '00951.TW', '3150.TW', '6957.TW', '00947.TW', '00949.TW']
    invalid += ['6838.TW', '00953B.TW', '00956.TW', '00954.TW']
    return list(set(tickers) - set(delisted) - set(invalid))

def rank(code, period='2y',  ticker_ref='^GSPC',
         rs_window='12mo',  out_dir='out'):
    tickers = get_tickers(code)
    #tickers = [t.lstrip('$') for t in tickers]
    tickers = remove_failed_tickers(tickers)

    rank_stock, rank_indust = ibd_rs.rankings(tickers, period=period,
                                              ticker_ref=ticker_ref,
                                              rs_window=rs_window)
    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    # Save to CSV
    print("\n\n***")
    os.makedirs(out_dir, exist_ok=True)
    today = datetime.now().strftime('%Y%m%d')
    for df, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_ibd{rs_window}_{today}.csv'
        #github.upload_df_as_csv(filename, df)
        df.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock, rank_indust

In [5]:
# @title Rank Function of IBD Financial Rating

from rs_rating import ibd_fin as fin


def rank_fin(code, out_dir='out'):
    tickers = get_tickers(code)
    #tickers = remove_failed_tickers(tickers)

    rank_stock = fin.financial_metric_ranking(tickers)
    if rank_stock.empty:
        print("Not enough data to generate rankings.")
        return

    # Save to CSV
    print("\n\n***")
    os.makedirs(out_dir, exist_ok=True)
    today = datetime.now().strftime('%Y%m%d')
    filename = f'{code}_stocks_fin_{today}.csv'
    #github.upload_df_as_csv(filename, rank_stock)
    rank_stock.to_csv(os.path.join(out_dir, filename), index=False)
    print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock

### Glossary of Terms

source (The source of stocks to analyze):
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector.  
  - For Taiwan Markets, possible values include:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所）
    - `TPEX`: Taipei Exchange （上櫃交易所）
    - `ESB`: Emerging Stock Board （興櫃交易所）
  - Can also be combined with '+' (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - For America Markets, possible values include:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index （費半指數）
  - Multiple indices can be combined using '+' (e.g., `SPX+DJIA+NDX+SOX`)

period (Historical Data Time Range)：
- The time range for which to fetch historical data.
- `2y` means 2 years
- `6mo` means 6 monthes

rs_window (Period for RS calculation)
- The period for Relative Strength calculation
- `3mo` means 3 months
- `12mo` means 12 months

RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
  - A value of 100 represents the performance of the benchmark index or market.
- The IBD RS calculates the performance of the last year, with the most recent quarter weighted double.
- The IBD 3-month RS calculates the performance of the last quarter

### RS Rating and Ranking

In [6]:
source = "S&P 500" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "Russell 1000", "Russell 2000", "PHLX Semiconductor", "U.S. Listed Stocks"]
period = "1y" # @param ["1y","ytd","2y"]
rs_window = "3mo" # @param ["12mo", "3mo"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'Russell 1000': 'RUI',
    'Russell 2000': 'RUT',
    'PHLX Semiconductor': 'SOX',
    'U.S. Listed Stocks': 'U.S.Listed',
}

rank_stock, rank_indust = rank(code_from_name[source], period,
                               rs_window=rs_window)
for df in (rank_stock, rank_indust):
    display(data_table.DataTable(df, include_index=False, num_rows_per_page=10))

[*********************100%***********************]  498 of 498 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AMTM']: YFInvalidPeriodError("%ticker%: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']")


[**********************100%**********************]  497 of 497 info downloaded


***
Your "SPX_stocks_1y_ibd3mo_20241005.csv" is in the "out" folder.
Your "SPX_industries_1y_ibd3mo_20241005.csv" is in the "out" folder.
***



,Ticker,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,Percentile (1M),Percentile (3M),Percentile (6M)
0,VST,138.41,Utilities,Utilities - Independent Power Producers,1271.42,-140.93,286.50,853.98,100.00,19.35,97.98,99.40
1,CEG,285.52,Utilities,Utilities—Renewable,948.21,-424.52,133.75,634.61,99.80,8.87,87.70,99.19
2,PLTR,40.01,Technology,Software - Infrastructure,700.11,863.16,289.75,231.35,99.60,96.77,98.39,80.65
3,WYNN,105.45,Consumer Cyclical,Resorts & Casinos,667.00,-402.68,-134.12,262.01,99.40,9.27,9.48,87.10
4,LVS,52.58,Consumer Cyclical,Resorts & Casinos,586.75,-238.64,-142.37,147.94,99.19,15.32,8.27,61.29
...,...,...,...,...,...,...,...,...,...,...,...,...
492,DLTR,70.90,Consumer Defensive,Discount Stores,-414.55,-3061.08,-175.47,-89.70,0.81,0.20,3.83,11.69
493,SMCI,41.23,Technology,Computer Hardware,-582.10,-2723.78,161.66,1024.29,0.60,0.40,92.14,100.00
494,MRNA,60.20,Healthcare,Biotechnology,-791.27,-2084.02,-127.35,155.51,0.40,1.01,10.28,63.31
495,HUM,240.03,Healthcare,Healthcare Plans,-855.16,261.98,77.70,-441.45,0.20,48.19,75.40,0.60


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,Percentile (1M),Percentile (3M),Percentile (6M)
0,Utilities—Renewable,Utilities,948.21,-424.52,133.75,634.61,CEG,100.00,8.59,89.84,99.22
1,Utilities - Independent Power Producers,Utilities,859.46,20.24,203.04,728.99,"VST,NRG",99.22,22.66,96.88,100.00
2,Resorts & Casinos,Consumer Cyclical,458.17,-259.41,-36.70,170.96,"WYNN,LVS,CZR,MGM",98.44,11.72,34.38,71.09
3,Airlines,Industrials,381.24,172.22,-9.18,42.55,"UAL,LUV,DAL",97.66,38.28,46.09,30.47
4,Engineering & Construction,Industrials,354.32,21.71,-29.58,285.84,"J,PWR",96.88,23.44,38.28,89.84
...,...,...,...,...,...,...,...,...,...,...,...
123,Discount Stores,Consumer Defensive,-94.94,-815.07,-22.48,100.52,"WMT,TGT,COST,DG,DLTR",3.91,1.56,42.19,46.09
124,Healthcare Plans,Healthcare,-116.72,255.53,-72.60,-93.13,"CVS,UNH,MOH,CI,CNC,ELV,HUM",3.12,47.66,20.31,5.47
125,Auto & Truck Dealerships,Consumer Cyclical,-151.10,369.99,35.45,141.00,KMX,2.34,67.19,66.41,63.28
126,Biotechnology,Healthcare,-180.74,-292.54,-4.67,-42.30,"INCY,TECH,VRTX,REGN,MRNA",1.56,10.16,48.44,12.50


### Financial (e.g., EPS, Revenue) RS Rating and Ranking

In [ ]:
source = "S&P 500" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "Russell 1000", "Russell 2000", "PHLX Semiconductor", "U.S. Listed Stocks"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'Russell 1000': 'RUI',
    'Russell 2000': 'RUT',
    'PHLX Semiconductor': 'SOX',
    'U.S. Listed Stocks': 'U.S.Listed',
}

df = rank_fin(code_from_name[source])
data_table.DataTable(df, include_index=False, num_rows_per_page=20)

[**********************100%**********************]  501 of 501 info downloaded
[**********************100%**********************]  501 of 501 financials downloaded
[**********************100%**********************]  501 of 501 financials downloaded




***
Your "SPX_stocks_fin_20241004.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,Price,EPS QoQ (%),QoQ 2Q Algo (%),QoQ 3Q Algo (%),EPS YoY (%),YoY 2Q Algo (%),EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE,EPS RS Rank (%),RPS RS Rank (%)
248,IRM,Real Estate,REIT—Specialty,118.43,-1.080000e+00,-0.12,-0.11,305.90,0.14,10166.12,0.78,-0.82,19.885,151.58,100.0,73.95
198,GE,Industrials,Aerospace & Defense,183.89,-2.200000e-01,-0.04,5.08,59.00,NaN,7133.63,3.70,-11.76,63.738,50.55,99.8,17.54
390,QRVO,Technology,Semiconductors,101.68,-5.580000e+00,46.99,-2.31,103.41,NaN,6762.55,-0.27,10.39,41.376,NaN,99.6,93.19
444,TRGP,Energy,Oil & Gas Midstream,155.50,9.000000e-02,0.12,0.13,-0.07,40.00,3091.15,4.74,1.29,73.064,33.33,99.4,79.66
447,TRV,Financial Services,Insurance - Property & Casualty,233.30,-1.100000e+00,-0.45,3.04,34.14,NaN,2260.61,15.79,-1.92,192.627,14.95,99.2,70.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,EMR,Industrials,Specialty Industrial Machinery,109.63,-5.000000e-01,2.48,-4.20,-27.21,NaN,-1789.86,2.96,-6.87,29.685,37.66,0.6,46.49
366,PFE,Healthcare,Drug Manufacturers—General,28.34,-5.400000e+01,2.09,-0.43,-40.00,NaN,-2818.48,-0.46,-6.09,9.757,NaN,0.4,52.10
20,ALB,Basic Materials,Specialty Chemicals,94.31,0.000000e+00,64.75,-3.04,-70.25,-133.12,-6603.50,-4.72,-39.02,63.514,NaN,0.2,1.00
201,GEV,Utilities,Utilities—Renewable,254.68,1.095000e+01,0.16,1.10,NaN,NaN,NaN,3.97,NaN,122.901,66.90,NaN,NaN
